# Introduction
The objective of this notebook is to show how to develop concept tests for unit tests and integration.

It is divided into the definition of functions, then the tests showing examples of bad practices, good practices with parameterization, brands, and finally integration tests.

# Setup

In [1]:
!pip install -U pytest

  Using cached pytest-7.4.0-py3-none-any.whl (323 kB)
  Using cached pluggy-1.2.0-py3-none-any.whl (17 kB)
  Using cached iniconfig-2.0.0-py3-none-any.whl (5.9 kB)
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
  Using cached exceptiongroup-1.1.2-py3-none-any.whl (14 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pytest --version

pytest 7.4.0


# Functions

### Functions to try

In [4]:
def procesar_nombre(nombre):
    if isinstance(nombre, str):
      return nombre.capitalize()
    return "Error nombre"

def procesar_apellido_paterno(apellido_p):
    if isinstance(apellido_p, str):
      return apellido_p.capitalize()
    return "Error Apellido Paterno"

def procesar_apellido_materno(apellido_m):
    if isinstance(apellido_m, str):
      return apellido_m.capitalize()
    return "Error Apellido Materno"

n = procesar_nombre("carlos")
ap = procesar_apellido_paterno("lopez")
am = procesar_apellido_materno("mejia")
print(n + ap + am)

CarlosLopezMejia


### Functions to save on file

The "magic command" of ipython `%%writefile file.py` is used to create a file with the content of the cell.

In [5]:
%%writefile funciones.py
def procesar_nombre(nombre):
    if isinstance(nombre, str):
      return nombre.capitalize()
    return "Error nombre"

def procesar_apellido_paterno(apellido_p):
    if isinstance(apellido_p, str):
      return apellido_p.capitalize()
    return "Error Apellido Paterno"

def procesar_apellido_materno(apellido_m):
    if isinstance(apellido_m, str):
      return apellido_m.capitalize()
    return "Error Apellido Materno"


Writing funciones.py


# Evidence

### Bad practices: multiple tests

In [6]:
%%writefile test_malas_practicas.py
import funciones as tf

# Pruebas para el nombre
def test_procesar_nombre():
  assert tf.procesar_nombre("carlos") == "Carlos"

def test_procesar_nombre_2():
  assert tf.procesar_nombre("MiGuel") == "Miguel"

def test_procesar_nombre_3():
  assert tf.procesar_nombre("iVAN") == "Ivan"

Writing test_malas_practicas.py


In [7]:
!pytest test_malas_practicas.py -v

============================= test session starts ==============================
platform darwin -- Python 3.10.12, pytest-7.4.0, pluggy-1.2.0 -- /Users/carlos/Downloads/ejemplo/venv/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/carlos/Downloads/ejemplo
configfile: pytest.ini
collected 3 items                                                              

test_malas_practicas.py::test_procesar_nombre PASSED                     [ 33%]
test_malas_practicas.py::test_procesar_nombre_2 PASSED                   [ 66%]
test_malas_practicas.py::test_procesar_nombre_3 PASSED                   [100%]

============================== 3 passed in 0.01s ===============================


### Good practices: parameterized tests

In [8]:
%%writefile test_buenas_practicas.py
import funciones as tf
import pytest

# Pruebas para el nombre
def obtener_datos_test_nombre():
        return [("carlos","Carlos"), ("MiGuel", "Miguel"), ("iVAN", "Ivan")]

@pytest.mark.parametrize('nombre, esperado', obtener_datos_test_nombre())
def test_nombre_parametrize(nombre, esperado):
        assert tf.procesar_nombre(nombre) == esperado

# Pruebas para el apellido paterno
def obtener_datos_test_ap():
        return [("LOPEZ","Lopez"), ("EspiNOZA", "Espinoza"), ("SmiTH", "Smith")]

@pytest.mark.parametrize('ap, esperado', obtener_datos_test_ap())
def test_ap_parametrize(ap, esperado):
        assert tf.procesar_apellido_paterno(ap) == esperado

# Pruebas para el apellido materno
def obtener_datos_test_am():
        return [("ferrer","Ferrer"), ("SILVa", "Silva"), ("PalafoX", "Palafox")]

@pytest.mark.parametrize('am, esperado', obtener_datos_test_am())
def test_am_parametrize(am, esperado):
        assert tf.procesar_apellido_paterno(am) == esperado

Writing test_buenas_practicas.py


In [9]:
!pytest test_buenas_practicas.py -v

============================= test session starts ==============================
platform darwin -- Python 3.10.12, pytest-7.4.0, pluggy-1.2.0 -- /Users/carlos/Downloads/ejemplo/venv/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/carlos/Downloads/ejemplo
configfile: pytest.ini
collected 9 items                                                              

test_buenas_practicas.py::test_nombre_parametrize[carlos-Carlos] PASSED  [ 11%]
test_buenas_practicas.py::test_nombre_parametrize[MiGuel-Miguel] PASSED  [ 22%]
test_buenas_practicas.py::test_nombre_parametrize[iVAN-Ivan] PASSED      [ 33%]
test_buenas_practicas.py::test_ap_parametrize[LOPEZ-Lopez] PASSED        [ 44%]
test_buenas_practicas.py::test_ap_parametrize[EspiNOZA-Espinoza] PASSED  [ 55%]
test_buenas_practicas.py::test_ap_parametrize[SmiTH-Smith] PASSED        [ 66%]
test_buenas_practicas.py::test_am_parametrize[ferrer-Ferrer] PASSED      [ 77%]
test_buenas_practicas.py::test_am_parametrize[SILVa-Silva] PASSED        

### Marks

In [10]:
%%writefile test_marks.py
import funciones as tf
import pytest

@pytest.mark.skip(reason="Check www.google.com. There is no way to try this now. ")
def test_calcular_curp():
    pass

#   xfail
@pytest.mark.xfail
def test_nombre_falla():
    assert tf.procesar_nombre(12) == "Doce"

#   Escribir un marcador personal
@pytest.mark.mi_marca
def test_mi_marca():
    assert tf.procesar_nombre("jorgE") == "Jorge"

@pytest.mark.xfail
def test_procesar_nombre_2():
  assert tf.procesar_nombre("MiGuel") == "Miguel"

Writing test_marks.py


We have to register our markers in a `pytest.ini`

In [11]:
%%writefile pytest.ini
[pytest]
markers =
    mi_marca: Ejemplo de marcador
    web_mark: Otro ejemplo de marcador

Overwriting pytest.ini


In [12]:
!pytest test_marks.py -v

============================= test session starts ==============================
platform darwin -- Python 3.10.12, pytest-7.4.0, pluggy-1.2.0 -- /Users/carlos/Downloads/ejemplo/venv/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/carlos/Downloads/ejemplo
configfile: pytest.ini
collected 4 items                                                              

test_marks.py::test_calcular_curp SKIPPED (Check www.google.com. The...) [ 25%]
test_marks.py::test_nombre_falla XFAIL                                   [ 50%]
test_marks.py::test_mi_marca PASSED                                      [ 75%]
test_marks.py::test_procesar_nombre_2 XPASS                              [100%]

============== 1 passed, 1 skipped, 1 xfailed, 1 xpassed in 0.05s ==============


In [13]:
!pytest --markers

@pytest.mark.mi_marca: Ejemplo de marcador

@pytest.mark.web_mark: Otro ejemplo de marcador

@pytest.mark.filterwarnings(warning): add a warning filter to the given test. see https://docs.pytest.org/en/stable/how-to/capture-warnings.html#pytest-mark-filterwarnings 

@pytest.mark.skip(reason=None): skip the given test function with an optional reason. Example: skip(reason="no way of currently testing this") skips the test.

@pytest.mark.skipif(condition, ..., *, reason=...): skip the given test function if any of the conditions evaluate to True. Example: skipif(sys.platform == 'win32') skips the test if we are on the win32 platform. See https://docs.pytest.org/en/stable/reference/reference.html#pytest-mark-skipif

@pytest.mark.xfail(condition, ..., *, reason=..., run=True, raises=None, strict=xfail_strict): mark the test function as an expected failure if any of the conditions evaluate to True. Optionally specify a reason for better reporting and run=False if you don't even want to exec

# Integration testing

In [14]:
%%writefile test_integracion.py
from funciones import procesar_nombre
from funciones import procesar_apellido_paterno
from funciones import procesar_apellido_materno
import pytest

def obtener_datos_test_integracion():
        return [("carlos","LOPEZ", "meJIa", "Carlos Lopez Mejia"),
                ("ivan", "huERTA", "CoroNA", "Ivan Huerta Corona")]

@pytest.mark.parametrize('nombre, ap, am, esperado', obtener_datos_test_integracion())
def test_divide_parametrize(nombre, ap, am, esperado):
        assert procesar_nombre(nombre) + " " + procesar_apellido_paterno(ap) + " " + procesar_apellido_materno(am) == esperado

Writing test_integracion.py


In [15]:
!pytest test_integracion.py -v

============================= test session starts ==============================
platform darwin -- Python 3.10.12, pytest-7.4.0, pluggy-1.2.0 -- /Users/carlos/Downloads/ejemplo/venv/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/carlos/Downloads/ejemplo
configfile: pytest.ini
collected 2 items                                                              

test_integracion.py::test_divide_parametrize[carlos-LOPEZ-meJIa-Carlos Lopez Mejia] PASSED [ 50%]
test_integracion.py::test_divide_parametrize[ivan-huERTA-CoroNA-Ivan Huerta Corona] PASSED [100%]

============================== 2 passed in 0.01s ===============================
